In [1]:
import os
import sys
import torch
import nilearn as nil
import nilearn.datasets
import nilearn.connectome
import nilearn.input_data
sys.path.append(os.path.join(".."))
import simexp_gcn
import simexp_gcn.data.raw_data_loader
import simexp_gcn.data.time_windows_dataset
import simexp_gcn.features.graph_construction
import simexp_gcn.models.yu_gcn

print("torch v{}".format(torch.__version__))
print("nilearn v{}".format(nil.__version__))
print("simexp_gcn v{}".format(simexp_gcn.__version__))

/home/ltetrel/.local/lib/python3.6/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


torch v1.8.1+cpu
nilearn v0.8.0
simexp_gcn v0.1.0


# Initial parameters

In [2]:
# parameters
num_parcels = 512 # number of parcels (nodes) to keep for the graph
window_length = 50 #number of timepoints per sample (timeserie)
zero_padding = True #pad splitted timeseries with zeros, or remove incomplete
random_seed = 0 # fix random state for dat ageneration, and pytorch fit
# paths
model_path = os.path.join("..", "models", "gcn_test.pt")
raw_dir = os.path.join("..", "data", "raw")
data_dir = os.path.join("..", "data", "processed", f"cobre_difumo{num_parcels}")
ts_out = os.path.join(data_dir, "timeseries")
conn_out = os.path.join(data_dir, "connectomes")

if not os.path.exists(ts_out):
    os.makedirs(ts_out)
if not os.path.exists(conn_out):
    os.makedirs(conn_out)

# Dataset fetching and connectomes generation

In [3]:
# Fetching atlas
atlas = nil.datasets.fetch_atlas_difumo(data_dir=raw_dir, dimension=num_parcels)
atlas_filename = atlas['maps']
atlas_labels = atlas['labels']

# Fetching data
data = nil.datasets.fetch_cobre(data_dir=raw_dir, n_subjects=None) #all subs

/home/ltetrel/.local/lib/python3.6/site-packages/numpy/lib/npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)
/home/ltetrel/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_cobre is deprecated; 'fetch_cobre' has been deprecated and will be removed in release 0.9 . Please consider using a different datasets or downloading it with a different tool than nilearn.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
# Estimating connectomes
# import numpy as np

# masker = nil.input_data.NiftiMapsMasker(maps_img=atlas['maps'], standardize=True, verbose=5)
# corr_measure = nil.connectome.ConnectivityMeasure(kind="correlation")

# for i in range(len(data.func)):
#     ts = masker.fit_transform(data.func[i], confounds=data.confounds[i])
#     conn = corr_measure.fit_transform([ts])[0]
#     sub_num = os.path.basename(data.func[i]).split('.')[0].split('_')[1]
#     np.save(os.path.join(ts_out, "timeseries_{}_difumo_{}.npy".format(sub_num, num_parcels)), ts)
#     np.save(os.path.join(conn_out, "conn_{}_difumo_{}.npy".format(sub_num, num_parcels)), conn)
#     print("Done {}/146".format(i+1))

# Data generator

In [5]:
# Preparing data for generator
# this include timeseries loading, splitting and labeling
pheno_path = os.path.join(raw_dir, "cobre", "phenotypic_data.tsv")
split_dir = os.path.join("..", "data", "interim")

In [6]:
# remove previous content
if os.path.exists(split_dir):
    import glob

    files = glob.glob(os.path.join(split_dir, "[^.gitkeep]*"))
    for f in files:
        os.remove(f)

In [7]:

RawDataLoad = simexp_gcn.data.raw_data_loader.RawDataLoader(
    num_nodes = num_parcels
    , ts_dir=ts_out
    , conn_dir=conn_out
    , pheno_path=pheno_path)
RawDataLoad.split_timeseries_and_save(window_length=window_length, zero_padding=zero_padding, output_dir=split_dir)

../simexp_gcn/data/raw_data_loader.py:103: UserWarning: Different shapes for sub ID(s): ['40075']
  warnings.warn("Different shapes for sub ID(s): {}".format(non_valid_ids))


In [8]:
# Pytorch dataset: generates items from the current data directory
train_dataset = simexp_gcn.data.time_windows_dataset.TimeWindowsDataset(
    data_dir=split_dir
    , partition="train"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True)
valid_dataset = simexp_gcn.data.time_windows_dataset.TimeWindowsDataset(
    data_dir=split_dir
    , partition="valid"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True)
test_dataset = simexp_gcn.data.time_windows_dataset.TimeWindowsDataset(
    data_dir=split_dir
    , partition="test"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True)
print("train dataset: {}".format(train_dataset))
print("valid dataset: {}".format(valid_dataset))
print("test dataset: {}".format(test_dataset))

train dataset: 304*(torch.Size([512, 50]), ())
valid dataset: 87*(torch.Size([512, 50]), ())
test dataset: 44*(torch.Size([512, 50]), ())


In [9]:
# Pytorch dataloader: wraps an iterable around the pytorch dataset to shuffle and generate (in parrallel) minibatches
#setting pytoch seed for reproducible torch.utils.data.DataLoader
torch.manual_seed(random_seed)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_generator = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_generator = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
train_features, train_labels = next(iter(train_generator))
print(f"Feature batch shape: {train_features.size()}; mean {torch.mean(train_features)}")
print(f"Labels batch shape: {train_labels.size()}; mean {torch.mean(torch.Tensor.float(train_labels))}")

Feature batch shape: torch.Size([16, 512, 50]); mean 1.210719302591201e-09
Labels batch shape: torch.Size([16]); mean 0.5625


# Model definition

In [10]:
# model definition
# get average connectome with its k-nearest neighbors
connectomes = RawDataLoad.get_valid_connectomes()
graph = simexp_gcn.features.graph_construction.make_group_graph(connectomes, k=8, self_loops=False, symmetric=True)
# Create model
gcn = simexp_gcn.models.yu_gcn.LoicGCN(graph.edge_index, graph.edge_attr, n_timepoints=window_length)
gcn

LoicGCN(
  (conv1): ChebConv(50, 32, K=2, normalization=sym)
  (conv2): ChebConv(32, 32, K=2, normalization=sym)
  (conv3): ChebConv(32, 16, K=2, normalization=sym)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

# Training

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch * len(X)
        correct = (pred.argmax(1) == y).type(torch.float).sum().item()
        correct /= X.shape[0]
        print(f"#{batch:>5};\ttrain_loss: {loss:>0.3f};\ttrain_accuracy: {(100*correct):>5.1f}%\t\t[{current:>5d}/{size:>5d}]")

def valid_test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model.forward(X)
            loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss /= size
    correct /= size

    return loss, correct

In [12]:
# Train and evaluate the model
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gcn.parameters(), lr=1e-4, weight_decay=5e-4)

epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}/{epochs}\n-------------------------------")
    train_loop(train_generator, gcn, loss_fn, optimizer)
    loss, correct = valid_test_loop(valid_generator, gcn, loss_fn)
    print(f"Valid metrics:\n\t avg_loss: {loss:>8f};\t avg_accuracy: {(100*correct):>0.1f}%")
print("Done!")

Epoch 1/15
-------------------------------
#    0;	train_loss: 0.826;	train_accuracy:  37.5%		[    0/  304]
#    1;	train_loss: 0.507;	train_accuracy:  81.2%		[   16/  304]
#    2;	train_loss: 1.492;	train_accuracy:  50.0%		[   32/  304]
#    3;	train_loss: 0.709;	train_accuracy:  56.2%		[   48/  304]
#    4;	train_loss: 0.819;	train_accuracy:  56.2%		[   64/  304]
#    5;	train_loss: 2.008;	train_accuracy:  31.2%		[   80/  304]
#    6;	train_loss: 1.182;	train_accuracy:  25.0%		[   96/  304]
#    7;	train_loss: 0.791;	train_accuracy:  56.2%		[  112/  304]
#    8;	train_loss: 1.291;	train_accuracy:  31.2%		[  128/  304]
#    9;	train_loss: 0.971;	train_accuracy:  62.5%		[  144/  304]
#   10;	train_loss: 1.031;	train_accuracy:  50.0%		[  160/  304]
#   11;	train_loss: 0.659;	train_accuracy:  56.2%		[  176/  304]
#   12;	train_loss: 0.793;	train_accuracy:  43.8%		[  192/  304]
#   13;	train_loss: 0.754;	train_accuracy:  50.0%		[  208/  304]
#   14;	train_loss: 0.866;	train_accuracy:  37.

# Test

In [13]:
loss, correct = valid_test_loop(test_generator, gcn, loss_fn)
print(f"Test metrics:\n\t avg_loss: {loss:>8f};\t avg_accuracy: {(100*correct):>0.1f}%")

Test metrics:
	 avg_loss: 0.078984;	 avg_accuracy: 38.6%


# Model saving

In [14]:
torch.save(gcn.state_dict(), model_path)

# PCA visualization

In [28]:
class GetActivation():
  def __init__(self, layer_name=None):
    self.layer_name = layer_name
    self.outputs = {}
      
  def __call__(self, module, module_in, module_out):
    self.outputs[self.layer_name] = module_out.detach()
      
  def clear(self):
    self.outputs = {}

In [43]:
import copy 

gcn_test = copy.deepcopy(gcn)
layer_name = "fc1"

# activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook
# gcn.fc3.register_forward_hook(get_activation(‘fc3’))
get_activation = GetActivation(layer_name)
for curr_layer_name, layer in gcn_test.named_modules():
  if curr_layer_name == layer_name:
    layer.register_forward_hook(get_activation)

In [44]:
gcn_test.eval()
for x, y in test_generator:
  gcn_test.fc1.register_forward_hook(get_activation)
  out = gcn_test.forward(x)
  print(get_activation.outputs)

AttributeError: 'LoicGCN' object has no attribute 'outputs'

In [ ]:
import sklearn
import matplotlib.pyplot as plt

Scaler = sklearn.preprocessing.StandardScaler()
PCA = sklearn.decomposition.PCA(n_components=2)

layer = "fc3.weight"
last_layer_parameters = gcn.state_dict()[layer].numpy()
# features (or samples for visualization) should be in y axis for sklearn
last_layer_parameters = last_layer_parameters.T
# normalization and pca decomposition
normalized_parameters = Scaler.fit_transform(last_layer_parameters)
pca_results = PCA.fit_transform(normalized_parameters)

fig, axes = plt.subplots(nrows=1, ncols=1)
axes.scatter(x=pca_results[:, 0], y=pca_results[:, 1])
axes.set_title("PCA deocmposition of layer {}".format(layer))
pca_results.shape

In [ ]:
Scaler = sklearn.preprocessing.StandardScaler()
PCA = sklearn.decomposition.PCA(n_components=2)

layer = "fc3.weight"
last_layer_parameters = gcn.state_dict()[layer].numpy()
# features (or samples for visualization) should be in y axis for sklearn
last_layer_parameters = last_layer_parameters.T
# normalization and pca decomposition
normalized_parameters = Scaler.fit_transform(last_layer_parameters)
pca_results = PCA.fit_transform(normalized_parameters)

fig, axes = plt.subplots(nrows=1, ncols=1)
axes.scatter(x=pca_results[:, 0], y=pca_results[:, 1])
axes.set_title("PCA deocmposition of layer {}".format(layer))
pca_results.shape